In [ ]:
import chess
import copy
#Library
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import os
import re
from tqdm import tqdm

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import matplotlib.patches as patches

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# from tensorflow.keras.layers.experimental import preprocessing
# from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from keras.applications.vgg16 import VGG16

import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
import os
import numpy as np
import sys

import random
# %matplotlib inline

In [4]:
#######################################################
modelDigitPath = "weights/DigitModel/DigitModel.pt"
modelCornerPath = "weights/CornerModel/CornerModel.pt"
modelPiecePath = "weights/PieceModel/piece.pt"
modelCenterPath = "weights/CenterModel/best.pt"
modelRCNNPath = "weights/RCNN/RCNN.pt"
videoPath = "/content/test_videos/"
vggClassifierPath = 'weights/classifierModel/model_weights.weights.h5'
pathList = ['2_Move_rotate_student.mp4', '2_move_student.mp4', '4_Move_studet.mp4', '6_Move_student.mp4','8_Move_student.mp4']
#######################################################

In [5]:
def createBoard():
    return [['-' for j in range(8)] for i in range(8)]

def printBoard(board):
    for i in range(8):
        for j in range(8):
            print(board[i][j],end = ' ')
        print('')

def toChessFormat(board):
    ans = ''
    c = 0;
    for i in range(8):
        for j in range(8):
            if (board[i][j] == '-'):
                c+=1
                if (j == 7):
                    ans += str(c)
                    c = 0

            else:
                if (c != 0):
                    ans += str(c)
                    c = 0
                ans += board[i][j]
        ans += '/'
    return ans[:-1]

def toOurFormat(chess):
    def isNumber(s):
        return (s >= '0' and s <= '9')
    ans = createBoard()
    i = 0
    j = 0
    for s in chess:
        if (isNumber(s)):
            j += int(s)
        elif (s == '/'):
            i += 1
            j = 0
        else:
            ans[i][j] = s
            j += 1
    return ans

def countPiece(board):
    count = 0
    for i in range(8):
        for j in range(8):
            if (board[i][j] != '-'):
                count += 1
    return count

"""
1. Find moved piece using before
2. Find list of changed using after
3.
"""

def notationToIndex(s):
    j = int(ord(s[0]) - 97) # a = 0
    i = 8 - int(s[1])
    return [i, j]

def indexToNotation(i,j):
    return chr(97 + j) + str(8 - i)

def pieceInNotation(s, pgn):
    if (s.upper() == 'P'): return pgn[0]
    return s.upper()

def genPossible(boardBefore, before, after):
    def isWhite(s):
        return s == s.upper()

    def checkLegal(beforeMatrix, mbefore ,mafter , white):
        if (not white):
            beforeMatrix = flip(beforeMatrix)
            x1 = notationToIndex(mbefore)
            x2 = notationToIndex(mafter)
            mbefore = indexToNotation(7-x1[0], 7-x1[1])
            mafter = indexToNotation(7-x2[0], 7-x2[1])
        chessFormat = toChessFormat(beforeMatrix)
        board = chess.Board(chessFormat)

        return mbefore + mafter in list(map(str,list(board.legal_moves)))

    bi, bj = notationToIndex(before)
    piece = boardBefore[bi][bj]
    black_white = isWhite(piece)
    # print(black_white)
    pieces = ['p','n','b','r','q','k']
    if (black_white):
        pieces = list(map(str.upper, pieces))

    notation = toChessFormat(boardBefore)
    if (checkLegal(boardBefore, before, after, black_white)):
        return True, pieceInNotation(piece, before), black_white

    tmp = copy.deepcopy(boardBefore)
    for piece in pieces:
        tmp[bi][bj] = piece
        notation = toChessFormat(tmp)
        if (checkLegal(tmp, before, after, black_white)):
            return True, pieceInNotation(piece, before), black_white

    return False, "Not Found", False

def difference(before, after):
    if countPiece(before) - countPiece(after) > 1:
        return False, "", False
    bef = ''
    af = []
    ate = []
    for i in range(8):
        for j in range(8):
            if (after[i][j] != before[i][j]):
                if (after[i][j] == '-'): #move from
                    bef = indexToNotation(i,j)
                else: #move to
                    af.append(indexToNotation(i,j))
                    if (before[i][j] == '-'): #Elim
                        ate.append('')
                    else:
                        ate.append('x')
    print(bef)
    if (bef):
        for i in range(len(af)):
            print(af[i])
            b, piece, white = genPossible(before,bef,af[i])
            if (b):

                ch = ''
                notation = toChessFormat(after)
                board = chess.Board(notation)
                if (white):
                    board = chess.Board(toChessFormat(flip(after)))
                if (board.is_check()):
                    ch = '+'
                if (not ate[i]):
                    if (piece in ['a','b','c','d','e','f','g','h']):
                        piece = ""
                return True, piece+ ate[i] + af[i]+ch, white

        return False, "", False
    return True, "", False

def flip(s):
    s = [[a for a in i][::-1] for i in s][::-1]
    ans = createBoard()
    for i in range(8):

      for j in range(8):
        if s[i][j]==s[i][j].upper():
            ans[i][j] = s[i][j].lower()
        elif s[i][j]==s[i][j].lower():
            ans[i][j] = s[i][j].upper()
    return ans

In [6]:
def readString(s):
    s= s.replace(" ", '')
    s = s.replace("\n", '')
    board = createBoard()
    i = 0
    j = 0
    for a in s:
        board[i][j] = a
        j += 1
        if (j == 8):
            i += 1
            j = 0

    return board

In [7]:
class RCNN:
  def __init__(self):
    pretrained_base = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
    pretrained_base.trainable = False
    self.classifier = tf.keras.Sequential([

    # BASE
    pretrained_base,

    # HEAD

    # Block 1
    layers.BatchNormalization(),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),

    # Block 2
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.GlobalMaxPooling2D(),
    layers.Dropout(0.4),

    # Output
    layers.BatchNormalization(),
    layers.Dense(12, activation='softmax'),
    ])
    self.classifier.load_weights(vggClassifierPath)
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.model = self.get_object_detection_model(13)
    checkpoint = torch.load(modelRCNNPath, map_location=self.device )
    self.model.to(self.device )
    self.model.load_state_dict(checkpoint['model_state_dict'])


  def get_object_detection_model(self,num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

  def apply_nms(self,orig_prediction, iou_thresh=0.3):

    keep = torchvision.ops.nms(orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)

    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]

    return final_prediction

  def to_piece(self,img,cor):
    return img[int(cor[1]):int(cor[3]),int(cor[0]):int(cor[2]),:].copy()

  def predict(self,img):
    ori = copy.deepcopy(img)
    img_res = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
    img_res /= 255.0
    img_res = torch.from_numpy(img_res).permute(2, 0, 1)
    img_res = img_res.to(torch.float32)
    self.model.eval()
    with torch.no_grad():
      prediction = self.model([img_res.to(self.device)])
    if(self.device == "cuda"):
      prediction = prediction.cpu()
    nms_prediction = self.apply_nms(prediction[0], iou_thresh=0.2)
    LABEL_NAMES = [
    'bg', 'white-queen', 'white-pawn', 'black-rook', 'black-bishop',
    'black-knight', 'black-queen', 'black-pawn', 'black-king',
    'white-rook', 'white-bishop', 'white-knight', 'white-king']
    result = []
    for box, label,clf in zip(nms_prediction['boxes'], nms_prediction['labels'],nms_prediction['scores']):
      x_min, y_min, x_max, y_max = box
      width, height = x_max - x_min, y_max - y_min
      piece = self.to_piece(np.array(ori),box)
      piece = cv2.resize(piece,(224,224))/255.
      pred = self.classifier.predict(np.array([piece]))[0]
      piece_name = LABEL_NAMES[np.argmax(pred)+1]
      result.append([[int(x_min), int(y_min), int(x_max), int(y_max)],piece_name,float(clf)])
    return result

In [8]:
class BoardAligner:
  def __init__(self):
    self.modelDigit = YOLO(modelDigitPath, verbose=False)
    self.modelCorner = YOLO(modelCornerPath, verbose=False)
    self.hasCorners = False

  def plotingLine(self):
    ptsDigit = {}
    self.canvas = np.zeros(self.img.shape[:-1], np.uint8)
    # cLine = 0
    for result in self.resultDigit:
        boxes = result.boxes  # Bounding box info
        for box in boxes:
            confidence = box.conf[0].item()  # Confidence score
            class_id = box.cls[0].item()  # Class ID
            xyxy = box.xyxy[0].tolist()  # [x_min, y_min, x_max, y_max]
            x_min, y_min, x_max, y_max = xyxy

            # Calculate the center of the box
            x_center = (x_min + x_max) / 2
            y_center = (y_min + y_max) / 2

            confidence = box.conf[0].item()  # Confidence score
            class_id = box.cls[0].item()  # Class ID
            if (class_id not in ptsDigit):
                ptsDigit[class_id] = (int(x_center),int(y_center))
            else:
                cv2.line(self.canvas, ptsDigit[class_id], (int(x_center),int(y_center)), 255 - class_id * 30, 5)
                # cLine+=1
            # ptsDigit.append([int(x_center),int(y_center), int(class_id)])
    # print("Line",cLine)
  def tranform(self):
    #Finding Corner Point
    pts1 = []
    for result in self.resultCorner:
      boxes = result.boxes  # Bounding box info
      for box in boxes:
          confidence = box.conf[0].item()  # Confidence score
          class_id = box.cls[0].item()  # Class ID
          xyxy = box.xyxy[0].tolist()  # [x_min, y_min, x_max, y_max]
          x_min, y_min, x_max, y_max = xyxy

          # Calculate the center of the box
          x_center = (x_min + x_max) / 2
          y_center = (y_min + y_max) / 2

          confidence = box.conf[0].item()  # Confidence score
          class_id = box.cls[0].item()  # Class ID
          pts1.append([int(x_center),int(y_center)])
    # print(pts1)
    assert len(pts1) == 4, "Can't Detect 4 corner"
    self.corners = pts1
    p = np.array(pts1)
    t1 = np.sum(p, axis=1)
    t2 = p[:,0] - p[:,1]
    inp = [p[np.argmin(t1)], p[np.argmin(t2)], p[np.argmax(t2)], p[np.argmax(t1)]]

    if(len(np.unique(inp, axis=0)) < len(inp)):
        t1 = p[:,1]
        t2 = p[:,0]
        inp = [p[np.argmin(t1)], p[np.argmax(t2)], p[np.argmin(t2)], p[np.argmax(t1)]]

    pts2 = np.float32([[0, 0],
                            [0, 1000 - 1],
                            [1000 - 1, 0],
                      [1000 - 1, 1000 - 1]])

    # Get the perspective transformation matrix
    self.M = cv2.getPerspectiveTransform(np.float32(inp), pts2)

    # Apply the perspective transformation
    self.dst = cv2.warpPerspective(self.img, self.M, (1000, 1000))
    self.lineOri = cv2.warpPerspective(self.canvas, self.M, (1000, 1000))

  def findOrient(self):
    l = []
    tmp = []
    v = True
    for i in self.lineOri[0]:
        if(i > 0):
            tmp.append(i)
            v = False
        if(len(tmp) > 0 and i == 0):
            m = max(tmp)
            l.append(m)
            tmp = []
        if(len(l) == 2):
            break
    if(len(l) >= 2):
        if (l[0] < l[1]):
            o = 3
        else:
            o = 1


    if(v):
        l = []
        tmp = []
        for i in self.lineOri[:,0]:
            if(i > 0):
                tmp.append(i)
                v = False
            if(len(tmp) > 0 and i == 0):
                m = max(tmp)
                l.append(m)
                tmp = []
            if(len(l) == 2):
                break
        if (len(l) >= 2):
            if (l[0] < l[1]):
                o = 4
            else:
                o = 2

    self.orient = o

    if o == 1:
        self.dst = cv2.rotate(self.dst, cv2.ROTATE_90_COUNTERCLOCKWISE)

    elif o == 2:
        self.dst = cv2.rotate(self.dst, cv2.ROTATE_180)

    elif o == 3:
        self.dst = cv2.rotate(self.dst, cv2.ROTATE_90_CLOCKWISE)


  def rotate_point(self,point):
    x, y = point
    image_shape = (1000,1000)
    height, width = image_shape[:2]

    if self.orient == 3:
        new_x = height - y
        new_y = x
    elif self.orient == 1:
        new_x = y
        new_y = width - x
    elif self.orient == 2:
        new_x = width - x
        new_y = height - y
    else:
        new_x = x
        new_y = y

    return new_x, new_y

  def predict(self,image):
    try:
        self.img = image.copy()
        if(not self.hasCorners):
          self.resultDigit = self.modelDigit.predict(image.copy())
          self.resultCorner = self.modelCorner.predict(image.copy())
        self.plotingLine()
        self.tranform()
        self.findOrient()
        if(len(self.corners) == 4):
          self.hasCorners = True
        else:
          self.hasCorners = False
        return True
    except:
      return False

In [9]:
class calculateOutput:
  def __init__(self):
    self.cheesModel = RCNN() #YOLO("PieceModel/piece.pt", verbose=False)
    self.centerModel = YOLO(modelCenterPath, verbose=False)
    self.aligner = BoardAligner()
    self.translator =  {
      'white-queen': 'Q',
      'white-pawn': 'P',
      'black-rook': 'r',
      'black-bishop': 'b',
      'black-knight': 'n',
      'black-queen': 'q',
      'black-pawn': 'p',
      'black-king': 'k',
      'white-rook': 'R',
      'white-bishop': 'B',
      'white-knight': 'N',
      'white-king': 'K'
    }

  def compute_iou(self,box1, box2):
    # Calculate the (x, y) coordinates of the intersection rectangle
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])

    # Check if there is an intersection
    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # Area of the intersection
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Areas of both rectangles
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Area of the union
    union_area = box1_area + box2_area - intersection_area

    # Compute IoU
    iou = intersection_area / union_area
    return iou

  def tranformToNewBoard(self):
    converted = []
    inp = self.pieces
    for i in inp:
      xm,ym,xma,yma = i[1]
      original_pointMi = np.array([xm, ym, 1])
      original_pointMa = np.array([xma, yma, 1])
      transformed_pointMi = np.dot(self.aligner.M, original_pointMi)
      transformed_pointMi /= transformed_pointMi[2]
      xm_new, ym_new = self.aligner.rotate_point(transformed_pointMi[:2])
      transformed_pointMa = np.dot(self.aligner.M, original_pointMa)
      transformed_pointMa /= transformed_pointMa[2]
      xma_new, yma_new = self.aligner.rotate_point(transformed_pointMa[:2])
      converted.append([i[0],[min(xm_new,xma_new),min(ym_new,yma_new),max(xm_new,xma_new),max(ym_new,yma_new)]])
    return converted

  def calCenter(self,point):
    return ((point[0]+point[2])//2,(point[1]+point[3])//2)

  def predictCenter(self):
    results = self.centerModel(self.aligner.dst.copy())
    self.center = []
    for result in results:
      boxes = result.boxes  # Bounding box info
      for box in boxes:
          class_id = box.cls[0].item()  # Class ID
          xyxy = box.xyxy[0].tolist()  # [x_min, y_min, x_max, y_max]
          self.center.append(xyxy)

  def assignCenter(self):
    self.predictCenter()
    lis = []
    for i in self.center:
      pred = sorted([[self.compute_iou(j[1],i) ,j[0]]for j in self.pieces])[-1]
      lis.append([pred[-1],i])
    self.center = lis

  def printOutput(self):
    o = [['-' for i in range(8)] for i in range(8)]
    for i in self.center:
      x,y = self.calCenter(i[1])
      # print(x,y,o)
      # print(self.translator[i[1]])
      o[int(y)//125][int(x)//125] = self.translator[i[0]]
    self.result = o
    for i in o:
      print(' '.join(i))

  def predict(self,image):
    # try:
      res = self.aligner.predict(image)
      if(not res):
        print("No")
        return False
      self.pieces = []
      results = self.cheesModel.predict(image.copy())
      for box in results: # Bounding box info
        xyxy = box[0] # [x_min, y_min, x_max, y_max]
        confidence = box[2]
        # if(len(self.pieces)>0):
        #   pred = sorted([[self.compute_iou(xyxy,i[1]),i ]for i in self.pieces])[-1]
        #   if(pred[0] >0.9):
        #     if(confidence>pred[1][-1]):
        #       self.pieces[self.pieces.index[pred[1]]] = [toName[class_id],xyxy,confidence]
        #     continue
        self.pieces.append([box[1],xyxy,confidence])
      self.pieces = self.tranformToNewBoard()
      self.assignCenter()
      self.printOutput()
      return True
    # except:
    #   return False

In [ ]:
ab = []
for vid in pathList:
    cap = cv2.VideoCapture(videoPath + vid,)
    test = calculateOutput()
    count =  0

    # Check if camera opened successfully
    if (cap.isOpened()== False):
        print("Error opening video stream or file")

    # Read until video is completed
    buffer = None
    pgnMove = []
    while(cap.isOpened()):
        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
            if(count%30 == 0):
                a = frame.copy()
                res = test.predict(a)
                if (res):
                    if (buffer):
                        b, ans, white= difference(copy.deepcopy(buffer), test.result)
                        print(ans)
                        if (ans):
                            pgnMove.append([ans,white])
                        if (b):
                            print("keep")
                            buffer = test.result
                    else :
                      buffer = test.result
                elif(not res):
                    print("no")
            count+=1
            key = cv2.waitKey(25)  # Wait indefinitely for a key press

            # If 'q' is pressed, break the loop
            if key == ord('q'):
                break

        # Break the loop
        else:
            break

    # When everything done, release the video capture object
    cap.release()
    ab.append(pgnMove)

In [ ]:
print(ab)

In [ ]:
pgnList = []
for i in ab:
  pgn = ""
  round = 1
  f = True
  for j in range(len(i)):
    if(f):
      pgn += str(round) + "."
      if(i[j][1]):
        pgn += " " + i[j][0] + " "
        f = False
      else:
        pgn += ".. " + i[j][0] + " "
        f = True
        round += 1
    else:
      if(not i[j][1]):
        pgn += i[j][0] + " "
        f = True
        round += 1
      else:
        round += 1
        pgn += ".. " + str(round) + ". " + i[j][0] + " "
        f = False
  pgn = pgn[:-1]
  pgnList.append(pgn)


data = {"row_id": pathList,
        "output": pgnList}
print(data)
import pandas as pd
df = pd.DataFrame(data)
df.to_csv('out.csv', index =False)

data